# Function

In [6]:
def SHA512(mess):

    # initialize hash values:
    # (first 64 bits of the fractional parts of the square roots of the first 8 primes 2..19):
    h0 = 0x6a09e667f3bcc908
    h1 = 0xbb67ae8584caa73b
    h2 = 0x3c6ef372fe94f82b
    h3 = 0xa54ff53a5f1d36f1
    h4 = 0x510e527fade682d1
    h5 = 0x9b05688c2b3e6c1f
    h6 = 0x1f83d9abfb41bd6b
    h7 = 0x5be0cd19137e2179
        
    # the round constants are based on the cube roots of the first 80 primes (2..4091):
    k = [0x428a2f98d728ae22, 0x7137449123ef65cd, 0xb5c0fbcfec4d3b2f, 0xe9b5dba58189dbbc, 0x3956c25bf348b538, 
            0x59f111f1b605d019, 0x923f82a4af194f9b, 0xab1c5ed5da6d8118, 0xd807aa98a3030242, 0x12835b0145706fbe, 
            0x243185be4ee4b28c, 0x550c7dc3d5ffb4e2, 0x72be5d74f27b896f, 0x80deb1fe3b1696b1, 0x9bdc06a725c71235, 
            0xc19bf174cf692694, 0xe49b69c19ef14ad2, 0xefbe4786384f25e3, 0x0fc19dc68b8cd5b5, 0x240ca1cc77ac9c65, 
            0x2de92c6f592b0275, 0x4a7484aa6ea6e483, 0x5cb0a9dcbd41fbd4, 0x76f988da831153b5, 0x983e5152ee66dfab, 
            0xa831c66d2db43210, 0xb00327c898fb213f, 0xbf597fc7beef0ee4, 0xc6e00bf33da88fc2, 0xd5a79147930aa725, 
            0x06ca6351e003826f, 0x142929670a0e6e70, 0x27b70a8546d22ffc, 0x2e1b21385c26c926, 0x4d2c6dfc5ac42aed, 
            0x53380d139d95b3df, 0x650a73548baf63de, 0x766a0abb3c77b2a8, 0x81c2c92e47edaee6, 0x92722c851482353b, 
            0xa2bfe8a14cf10364, 0xa81a664bbc423001, 0xc24b8b70d0f89791, 0xc76c51a30654be30, 0xd192e819d6ef5218, 
            0xd69906245565a910, 0xf40e35855771202a, 0x106aa07032bbd1b8, 0x19a4c116b8d2d0c8, 0x1e376c085141ab53, 
            0x2748774cdf8eeb99, 0x34b0bcb5e19b48a8, 0x391c0cb3c5c95a63, 0x4ed8aa4ae3418acb, 0x5b9cca4f7763e373, 
            0x682e6ff3d6b2b8a3, 0x748f82ee5defb2fc, 0x78a5636f43172f60, 0x84c87814a1f0ab72, 0x8cc702081a6439ec, 
            0x90befffa23631e28, 0xa4506cebde82bde9, 0xbef9a3f7b2c67915, 0xc67178f2e372532b, 0xca273eceea26619c, 
            0xd186b8c721c0c207, 0xeada7dd6cde0eb1e, 0xf57d4f7fee6ed178, 0x06f067aa72176fba, 0x0a637dc5a2c898a6, 
            0x113f9804bef90dae, 0x1b710b35131c471b, 0x28db77f523047d84, 0x32caab7b40c72493, 0x3c9ebe0a15c9bebc, 
            0x431d67c49c100d4c, 0x4cc5d4becb3e42b6, 0x597f299cfc657e2a, 0x5fcb6fab3ad6faec, 0x6c44198c4a475817]


    hash_constants = [h0, h1, h2, h3, h4, h5, h6, h7]


    def rotate_right(number, amount):
        number = number & 0xFFFFFFFFFFFFFFFF
        return (number >> amount) | (number << (64 - amount)) & 0xFFFFFFFFFFFFFFFF


    def shift_right(number, amount):
        return number >> amount
 

    def hashing(mess):
        h0, h1, h2, h3, h4, h5, h6, h7 = hash_constants
        # object bytearray keeps same length of the original string
        # https://docs.python.org/3/library/stdtypes.html#bytearray
        mess = bytearray(mess, encoding='utf8')
        # message length in bits
        mess_length = (len(mess) * 8) & 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF
        # message length in bytes
        # https://docs.python.org/3/library/stdtypes.html#int.to_bytes
        mess_length_bytes = mess_length.to_bytes(8, byteorder='big')
        # append single '1' bit
        mess.append(0x80)
        # append K zeros so that after adding the 64 bit length the message 
        # will be 512 (or a multiple) bit long
        while (len(mess) + len(mess_length_bytes)) % 128 != 0:
            mess.append(0)
        # adding the 64 bit length of the original message
        mess += mess_length_bytes
        assert len(mess) % 128 == 0, f"Padding error! Length of message is: {len(mess)}"
        # process the message in successive 1024-bit = 128-bytes chunks:
        for chunk_range in range(0, len(mess), 128):
            chunks = mess[chunk_range : chunk_range + 128]
            w = [0 for i in range(128)]
            # break chunk into sixteen 64-bit = 8-bytes words w[i], 0 ≤ i ≤ 15
            for i in range(16):
                w[i] = int.from_bytes(chunks[8*i : 8*i + 8], byteorder='big')
            # extend the first 16 words into the remaining 64
            # words w[16..79] of the message schedule array:
            for i in range(16, 80):
                s0 = (rotate_right(w[i-15], 1) ^ rotate_right(w[i-15], 8) ^ shift_right(w[i-15], 7)) & 0xFFFFFFFFFFFFFFFF
                s1 = (rotate_right(w[i-2], 19) ^ rotate_right(w[i-2], 61) ^ shift_right(w[i-2], 6)) & 0xFFFFFFFFFFFFFFFF
                w[i] = (w[i-16] + s0 + w[i-7] + s1) & 0xFFFFFFFFFFFFFFFF
            # initialize variables to current hash value
            a, b, c, d, e, f, g, h = h0, h1, h2, h3, h4, h5, h6, h7
            # compression function (main loop)
            for i in range(80):
                S1 = (rotate_right(e, 14) ^ rotate_right(e, 18) ^ rotate_right(e, 41)) & 0xFFFFFFFFFFFFFFFF
                ch = ((e & f) ^ ((~e) & g)) & 0xFFFFFFFFFFFFFFFF
                temp1 = (h + S1 + ch + k[i] + w[i]) & 0xFFFFFFFFFFFFFFFF
                S0 = (rotate_right(a, 28) ^ rotate_right(a, 34) ^ rotate_right(a, 39)) & 0xFFFFFFFFFFFFFFFF
                maj = ((a & b) ^ (a & c) ^ (b & c)) & 0xFFFFFFFFFFFFFFFF
                temp2 = (S0 + maj) & 0xFFFFFFFFFFFFFFFF
                
                a_tmp = (temp1 + temp2) & 0xFFFFFFFFFFFFFFFF
                e_tmp = (d + temp1) & 0xFFFFFFFFFFFFFFFF
                # rotate the 8 variables
                a, b, c, d, e, f, g, h = a_tmp, a, b, c, e_tmp, e, f, g

            # add this chunk's hash to the result:
            h0 = (h0 + a) & 0xFFFFFFFFFFFFFFFF
            h1 = (h1 + b) & 0xFFFFFFFFFFFFFFFF
            h2 = (h2 + c) & 0xFFFFFFFFFFFFFFFF
            h3 = (h3 + d) & 0xFFFFFFFFFFFFFFFF
            h4 = (h4 + e) & 0xFFFFFFFFFFFFFFFF
            h5 = (h5 + f) & 0xFFFFFFFFFFFFFFFF
            h6 = (h6 + g) & 0xFFFFFFFFFFFFFFFF
            h7 = (h7 + h) & 0xFFFFFFFFFFFFFFFF

        output_hash = [h0, h1, h2, h3, h4, h5, h6, h7]
        
        return ''.join(hex(x)[2:].zfill(16) for x in output_hash)

      
    return hashing(mess)

# Use here!

In [5]:
your_hash = input('Message you want hashed: ')
print('Hashed message: ', SHA512(your_hash))

Hashed message:  ee26b0dd4af7e749aa1a8ee3c10ae9923f618980772e473f8819a5d4940e0db27ac185f8a0e1d5f84f88bc887fd67b143732c304cc5fa9ad8e6f57f50028a8ff


# Test

In [13]:
import string, random, hashlib

def test(n, m):
    # creates random strings and hashes them with both my implementation and hashlib's implementation
    def random_string(n):
        return ''.join(random.choice(string.ascii_letters + string.digits) for _ in range(n))

    for _ in range(n):
        random_s = random_string(m)
        assert hashlib.sha512(bytes(random_s, encoding='utf8')).hexdigest() == SHA512(random_s), f"Error in hash for {random_s}!"

test(1000, 1000)